In [1]:
import os
import sys
from pathlib import Path
django_project_dir = Path(".")
sys.path.insert(0, str(django_project_dir))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "newsilkroad.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

[raven.contrib.django.client.DjangoClient]DEBUG 08:50:37 Configuring Raven for host: None
[raven.contrib.django.client.DjangoClient]INFO 08:50:37 Raven is not configured (logging is disabled). Please see the documentation for more information.


In [2]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
from infrastructure.models import Project, PowerPlant
from locations.models import GeometryStore
import ipdb
from django.db import connection
from shapely.geometry import Point, LineString, Polygon

### Get Geodata objects from the GeometryStore via Projects

In [3]:
sql_lines = Project.objects.filter(geo__isnull=False).filter(geo__lines__geom__isnull=False).values("identifier", "geo__lines__attributes", "geo__lines__geom")
sql_points = Project.objects.filter(geo__isnull=False).filter(geo__points__geom__isnull=False).values("identifier", "geo__points__attributes", "geo__points__geom")

df_lines = geopandas.read_postgis(str(sql_lines.query), connection)
df_points = geopandas.read_postgis(str(sql_points.query), connection)

### Cast UUIDs to String before exporting to GeoJSON

In [4]:
df_lines['identifier'] = df_lines['identifier'].apply(str)
df_lines.to_file("geojson/reconasia-lines.geojson", driver="GeoJSON")

In [5]:
df_points['identifier'] = df_points['identifier'].apply(str)
df_points.to_file("geojson/reconasia-points.geojson", driver="GeoJSON")

### Get the  Project View table

In [6]:
project_view = pd.read_sql("SELECT * FROM infrastructure_projects_export_view;", connection)
project_view['identifier'] = project_view['identifier'].apply(str)
project_view.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14175 entries, 0 to 14174
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   identifier                     14175 non-null  object 
 1   name                           14175 non-null  object 
 2   infrastructure_type            14169 non-null  object 
 3   countries                      14172 non-null  object 
 4   regions                        14120 non-null  object 
 5   contractors                    3959 non-null   object 
 6   initiatives                    1156 non-null   object 
 7   operators                      754 non-null    object 
 8   funding_sources                1547 non-null   object 
 9   funding_amounts                1547 non-null   object 
 10  funding_currencies             1547 non-null   object 
 11  fuel_type                      10970 non-null  object 
 12  fuel_category                  10970 non-null 

In [7]:
gs = Project.objects.filter(geo__isnull=False).filter(geo__centroid__isnull=False).values("identifier", "geo__centroid")
centroids = pd.DataFrame(gs)
centroids['centroid'] = centroids['geo__centroid'].apply(Point)
centroids['identifier'] = centroids['identifier'].apply(str)
centroids = centroids.drop(columns=["geo__centroid"])

### Merge the centroids into the project view data

In [8]:
replaced = project_view.set_index('identifier').join(centroids.set_index('identifier'))

```NOTE: Turning the identifier into an index removes it from the info() list on the replaced dataframe. Giving you 61 columns before and after the join```

In [9]:
geo_replaced = geopandas.GeoDataFrame(replaced, geometry='centroid')
geo_replaced.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 14175 entries, 1de2c76a-9775-43a4-9d9f-c7cf61538d17 to 2e77d1d3-2dcf-4e3c-b256-81bb27f3268e
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   name                           14175 non-null  object  
 1   infrastructure_type            14169 non-null  object  
 2   countries                      14172 non-null  object  
 3   regions                        14120 non-null  object  
 4   contractors                    3959 non-null   object  
 5   initiatives                    1156 non-null   object  
 6   operators                      754 non-null    object  
 7   funding_sources                1547 non-null   object  
 8   funding_amounts                1547 non-null   object  
 9   funding_currencies             1547 non-null   object  
 10  fuel_type                      10970 non-null  object  
 11  fuel_category           

In [10]:
geo_replaced.columns.difference(project_view.columns)

Index(['centroid'], dtype='object')

In [11]:
project_view.columns.difference(geo_replaced.columns)

Index(['identifier'], dtype='object')

In [12]:
geo_replaced

,name,infrastructure_type,countries,regions,contractors,initiatives,operators,funding_sources,funding_amounts,funding_currencies,...,pipeline_metered,pipeline_throughput,pipeline_throughput_unit,pipeline_throughput_timeframe,pipeline_throughput_year,design_voltage,direct_current,electricity_flow,estimated_transfer_capacity,centroid
identifier,,,,,,,,,,,,,,,,,,,,,
1de2c76a-9775-43a4-9d9f-c7cf61538d17,North-South Road Transport Corridor (Phase 4),Road,'Armenia',"'Russia, Central Asia and the South Caucasus'",None,'International North-South Transport Corridor',None,'Eurasian Development Bank' 'European Investme...,150000000 NULL,USD USD,...,None,NaN,,,None,NaN,None,None,NaN,None
9d46759b-4222-42f6-a5b7-b0c4430f38e7,M6 Vanadzor-Alaverdi-Georgia Interstate Road R...,Road,'Armenia',"'Russia, Central Asia and the South Caucasus'",None,None,None,'Asian Development Bank (ADB)' 'Armenia' 'Euro...,225000 50000000 51000000,USD USD EUR,...,None,NaN,,,None,NaN,None,None,NaN,None
a0050ef7-7d37-4a41-b4e8-9d88a27f1d7d,East-West Highway Improvement,Road,'Azerbaijan',"'Russia, Central Asia and the South Caucasus'","'Impresa S.p.A.', 'Polat Yol Yapı San. ve Tic....",'Transport Corridor Europe-Caucasus-Asia (TRAC...,'Azeravtoyol OJSC','Azerbaijan' 'Asian Development Bank (ADB)' 'S...,73800000 40400000 13000000 10400000,USD USD USD USD,...,None,NaN,,,None,NaN,None,None,NaN,POINT (46.22690 40.86572)
729efb76-3a6c-4879-871e-45b9685821df,Highway 2,Road,'Azerbaijan',"'Russia, Central Asia and the South Caucasus'","'Makyol İnşaat A.Ş.', 'KOLIN Construction, Tou...",'Azerbaijan State Programme on Poverty Reducti...,None,'Azerbaijan' 'International Bank for Reconstru...,64870000 200000000,USD USD,...,None,NaN,,,None,NaN,None,None,NaN,POINT (48.87727 40.39947)
efca2104-5465-4e13-a869-e23d96faf30b,Baku-Tbilisi-Kars Railway,Rail,"'Azerbaijan', 'Georgia', 'Turkey'","'Russia, Central Asia and the South Caucasus'",'Özgün Yapı & Çelikler Consortium (Kars-Akhalk...,"'Lapis Lazuli Route', 'Silk Wind', 'Azerbaijan...",None,'Azerbaijan' 'Georgia',775000000 79440000,USD USD,...,None,NaN,,,None,NaN,None,None,NaN,POINT (46.21902 40.85021)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e396bbec-564f-4b08-a0d2-b967a8a8e63d,Vukosavlje-Doboj Highway Project (Vukosavlje-B...,Road,'Bosnia and Herzegovina','Eastern Europe and Balkans',None,"'CEEC ""16+1”', 'Belt and Road'",None,None,None,None,...,None,NaN,,,None,NaN,None,None,NaN,POINT (18.04077 45.36766)
c1f67b80-deb2-4cb6-8daf-295fa4784638,"Modernization of Rijeka-Zagreb Rail, Section K...",Rail,'Croatia','Eastern Europe and Balkans','China Road and Bridge Corporation',None,None,None,None,None,...,None,NaN,,,None,NaN,None,None,NaN,POINT (15.37388 45.32569)
aeaece46-19b0-4588-8562-f6793524ff12,"Sumsel-5 Coal-fired Power Plant, South Sumatra",None,None,None,None,None,None,None,None,None,...,None,NaN,,,None,NaN,None,None,NaN,POINT (103.75455 -2.15618)


In [29]:
geo_replaced['longitude'] = geo_replaced['centroid'].x
geo_replaced['latitude'] = geo_replaced['centroid'].y
geo_replaced

,plant_name,plant_countries,plant_regions,plant_operators,plant_owners,plant_owners_stake,plant_slug,plant_status,plant_sources,plant_total_cost,...,plant_CO2_emissions,plant_co2_emissions_unit,plant_grid_connected,plant_description,plant_created_at,plant_updated_at,plant_published,centroid,latitude,longitude
plant_id,,,,,,,,,,,,,,,,,,,,,
1,Elbasan Waste to Energy Power Plant,'Albania','Eastern Europe and Balkans',None,None,None,elbasan-waste-to-energy-power-plant,Active,,NaN,...,NaN,NULL,None,,2018-09-14 20:08:52.510035+00:00,2018-09-14 20:08:52.764026+00:00,True,POINT (20.09568 41.10858),41.108580,20.095680
3,AVN Waste Incineration Plant II,'Austria','Western Europe and Scandinavia',None,'Abfallverwertung Niederosterreich GmbH',NULL,avn-waste-incineration-plant-ii,Active,,NaN,...,NaN,NULL,None,,2018-09-14 20:08:53.487785+00:00,2018-09-14 20:08:53.959625+00:00,True,None,NaN,NaN
5,Saalfelden Biomass Power Plant,'Austria','Western Europe and Scandinavia',None,None,None,saalfelden-biomass-power-plant,Active,,NaN,...,NaN,NULL,None,,2018-09-14 20:08:54.783561+00:00,2018-09-14 20:08:54.887968+00:00,True,None,NaN,NaN
9,Modling Biomass Power Plant,'Austria','Western Europe and Scandinavia','EVN AG',None,None,modling-biomass-power-plant,Active,,NaN,...,225000.0,Tonnes per annum,None,,2018-09-14 20:08:57.396882+00:00,2018-09-14 20:08:57.588782+00:00,True,POINT (16.28440 48.08589),48.085890,16.284400
12,Heiligenkreuz Biomass Power Plant,'Austria','Western Europe and Scandinavia',None,None,None,heiligenkreuz-biomass-power-plant,Active,,NaN,...,NaN,NULL,None,,2018-09-14 20:08:58.813704+00:00,2018-09-14 20:08:58.974353+00:00,True,POINT (16.25845 46.98874),46.988740,16.258450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13917,Loznica Natural Gas Power Plant,'Serbia','Eastern Europe and Balkans',None,'Scarborough Group',NULL,loznica-natural-gas-power-plant,Inactive,,2.300000e+08,...,NaN,NULL,None,,2020-03-31 16:05:45.821099+00:00,2020-03-31 16:05:45.821180+00:00,True,None,NaN,NaN
13918,Northeast Serbia Wind Park,'Serbia','Eastern Europe and Balkans',None,None,None,northeast-serbia-wind-park,Inactive,,NaN,...,NaN,NULL,None,,2020-03-31 19:54:54.472973+00:00,2020-03-31 19:54:54.473004+00:00,True,None,NaN,NaN
13919,Stavalj Thermal Power Plant,'Serbia','Eastern Europe and Balkans',None,None,None,stavalj-thermal-power-plant,Inactive,,NaN,...,NaN,NULL,None,,2020-03-31 20:41:08.314943+00:00,2020-03-31 20:41:08.314974+00:00,True,None,NaN,NaN


In [37]:
geo_replaced.to_file("geojson/reconasia-projects.geojson", driver="GeoJSON")
geo_replaced.to_csv("geojson/reconasia-projects.csv")

### Get the Power Plant View table

In [38]:
plant_view = pd.read_sql("SELECT * FROM infrastructure_powerplant_export_view;", connection)
plant_view.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7241 entries, 0 to 7240
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   plant_id                     7241 non-null   int64              
 1   plant_name                   7241 non-null   object             
 2   plant_countries              6927 non-null   object             
 3   plant_regions                6924 non-null   object             
 4   plant_operators              2486 non-null   object             
 5   plant_owners                 5493 non-null   object             
 6   plant_owners_stake           5493 non-null   object             
 7   plant_slug                   7241 non-null   object             
 8   plant_status                 7241 non-null   object             
 9   plant_sources                7241 non-null   object             
 10  plant_total_cost             43 non-null     flo

In [39]:
gs = PowerPlant.objects.filter(geo__isnull=False).filter(geo__centroid__isnull=False).values("id", "geo__centroid")
centroids = pd.DataFrame(gs)
centroids['centroid'] = centroids['geo__centroid'].apply(Point)
centroids = centroids.drop(columns=["geo__centroid"])
centroids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4972 entries, 0 to 4971
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        4972 non-null   int64 
 1   centroid  4972 non-null   object
dtypes: int64(1), object(1)
memory usage: 77.8+ KB


In [40]:
replaced = plant_view.set_index('plant_id').join(centroids.set_index('id'))

In [41]:
geo_replaced = geopandas.GeoDataFrame(replaced, geometry='centroid')
geo_replaced.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 7241 entries, 1 to 13921
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   plant_name                   7241 non-null   object             
 1   plant_countries              6927 non-null   object             
 2   plant_regions                6924 non-null   object             
 3   plant_operators              2486 non-null   object             
 4   plant_owners                 5493 non-null   object             
 5   plant_owners_stake           5493 non-null   object             
 6   plant_slug                   7241 non-null   object             
 7   plant_status                 7241 non-null   object             
 8   plant_sources                7241 non-null   object             
 9   plant_total_cost             43 non-null     float64            
 10  plant_total_cost_currency    7241 non-n

In [42]:
geo_replaced['latitude'] = geo_replaced['centroid'].y
geo_replaced['longitude'] = geo_replaced['centroid'].x

In [43]:
geo_replaced.to_file("geojson/reconasia-plants.geojson", driver="GeoJSON")
geo_replaced.to_csv("geojson/reconasia-plants.csv")

In [44]:
! ls -lhR geojson/

geojson/:
total 52M
drwxr-xr-x 3 jgibson jgibson 4.0K Nov 25 15:37 lines
drwxr-xr-x 3 jgibson jgibson 4.0K Nov 25 15:37 points
drwxr-xr-x 3 jgibson jgibson 4.0K Nov 25 17:37 projects
-rw-r--r-- 1 jgibson jgibson  30M Dec  7 08:50 reconasia-lines.geojson
-rw-r--r-- 1 jgibson jgibson 2.1M Dec  7 09:08 reconasia-plants.csv
-rw-r--r-- 1 jgibson jgibson 8.2M Dec  7 09:08 reconasia-plants.geojson
-rw-r--r-- 1 jgibson jgibson 1.7M Dec  7 08:50 reconasia-points.geojson
-rw-r--r-- 1 jgibson jgibson 2.1M Dec  7 09:06 reconasia-projects.csv
-rw-r--r-- 1 jgibson jgibson 8.2M Dec  7 09:06 reconasia-projects.geojson

geojson/lines:
total 30M
-rw-r--r-- 1 jgibson jgibson 30M Nov 25 15:37 lines.geojson

geojson/points:
total 1.7M
-rw-r--r-- 1 jgibson jgibson 1.7M Nov 25 15:37 points.geojson

geojson/projects:
total 26M
-rw-r--r-- 1 jgibson jgibson 26M Nov 25 17:37 projects.geojson


In [26]:
geo_replaced

,plant_name,plant_countries,plant_regions,plant_operators,plant_owners,plant_owners_stake,plant_slug,plant_status,plant_sources,plant_total_cost,...,plant_CO2_emissions,plant_co2_emissions_unit,plant_grid_connected,plant_description,plant_created_at,plant_updated_at,plant_published,centroid,latitude,longitude
plant_id,,,,,,,,,,,,,,,,,,,,,
1,Elbasan Waste to Energy Power Plant,'Albania','Eastern Europe and Balkans',None,None,None,elbasan-waste-to-energy-power-plant,Active,,NaN,...,NaN,NULL,None,,2018-09-14 20:08:52.510035+00:00,2018-09-14 20:08:52.764026+00:00,True,POINT (20.09568 41.10858),20.095680,41.108580
3,AVN Waste Incineration Plant II,'Austria','Western Europe and Scandinavia',None,'Abfallverwertung Niederosterreich GmbH',NULL,avn-waste-incineration-plant-ii,Active,,NaN,...,NaN,NULL,None,,2018-09-14 20:08:53.487785+00:00,2018-09-14 20:08:53.959625+00:00,True,None,NaN,NaN
5,Saalfelden Biomass Power Plant,'Austria','Western Europe and Scandinavia',None,None,None,saalfelden-biomass-power-plant,Active,,NaN,...,NaN,NULL,None,,2018-09-14 20:08:54.783561+00:00,2018-09-14 20:08:54.887968+00:00,True,None,NaN,NaN
9,Modling Biomass Power Plant,'Austria','Western Europe and Scandinavia','EVN AG',None,None,modling-biomass-power-plant,Active,,NaN,...,225000.0,Tonnes per annum,None,,2018-09-14 20:08:57.396882+00:00,2018-09-14 20:08:57.588782+00:00,True,POINT (16.28440 48.08589),16.284400,48.085890
12,Heiligenkreuz Biomass Power Plant,'Austria','Western Europe and Scandinavia',None,None,None,heiligenkreuz-biomass-power-plant,Active,,NaN,...,NaN,NULL,None,,2018-09-14 20:08:58.813704+00:00,2018-09-14 20:08:58.974353+00:00,True,POINT (16.25845 46.98874),16.258450,46.988740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13917,Loznica Natural Gas Power Plant,'Serbia','Eastern Europe and Balkans',None,'Scarborough Group',NULL,loznica-natural-gas-power-plant,Inactive,,2.300000e+08,...,NaN,NULL,None,,2020-03-31 16:05:45.821099+00:00,2020-03-31 16:05:45.821180+00:00,True,None,NaN,NaN
13918,Northeast Serbia Wind Park,'Serbia','Eastern Europe and Balkans',None,None,None,northeast-serbia-wind-park,Inactive,,NaN,...,NaN,NULL,None,,2020-03-31 19:54:54.472973+00:00,2020-03-31 19:54:54.473004+00:00,True,None,NaN,NaN
13919,Stavalj Thermal Power Plant,'Serbia','Eastern Europe and Balkans',None,None,None,stavalj-thermal-power-plant,Inactive,,NaN,...,NaN,NULL,None,,2020-03-31 20:41:08.314943+00:00,2020-03-31 20:41:08.314974+00:00,True,None,NaN,NaN
